In [2]:
import os
import glob

path = "Ads_problem4" # Path of the folder where the images for the Q4 are present
all_dir = os.listdir(path) # Getting all the subdirectories present in the Ads_problem4 folder
data = {}

all_img = []

for i in glob.iglob("Ads_problem4/*/"): # Iterating through all the sub-folders in the Ads_problem folder
    data[i] = None
    d = {}
    
    for j in glob.iglob(f"{i}/*/"): # Iterating through all the folders in the subfolders
        
        d[j] = []
        for k in glob.iglob(f"{j}/*"):# Iterating through all the images in the subfolders
            if not(k.endswith("csv")):
                d[j].append(k)
                all_img.append(k) # Appending all the images to the all_img list
    data[i] = d



{'Ads_problem4\\Ramen\\Ideal\\': ['Ads_problem4\\Ramen\\Ideal\\AdobeStock_254640691.jpeg', 'Ads_problem4\\Ramen\\Ideal\\AdobeStock_255497590.jpeg'], 'Ads_problem4\\Ramen\\Other\\': ['Ads_problem4\\Ramen\\Other\\AdobeStock_175017150.jpeg', 'Ads_problem4\\Ramen\\Other\\AdobeStock_288691583.jpeg', 'Ads_problem4\\Ramen\\Other\\AdobeStock_297274410.jpeg']}


In [3]:
from ultralytics import YOLO
import os
import numpy as np


def count_entities(results):
        d= {}
        coord = []
        names = model.names
        
        for r in results:
            coord = (r.boxes.xywh).tolist()
            for c in r.boxes.cls:
                key = names[int(c)]
                d[key] = d.get(key, 0)+1
        last = 0
        for k,v in d.items():
            d[k] = list(str(v)) 
            for c in range(v):
                d[k] += [coord[last + c]]
                if c == v-1:
                    last +=c + 1
                    
        
        return d




model = YOLO("yolov8x.pt")
names = model.names
entity = {}

for f in all_img:
    results = model.predict(source = f, conf = 0.5,stream=True)
    cnt = count_entities(results)
    entity[f] = cnt

print(entity)


image 1/1 C:\Users\aliat\Downloads\Aithon\Ads_problem4\Pizza\Ideal\AdobeStock_323628715.jpeg: 448x640 3 persons, 3 cups, 4 pizzas, 1 dining table, 3710.3ms
Speed: 9.6ms preprocess, 3710.3ms inference, 10.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 C:\Users\aliat\Downloads\Aithon\Ads_problem4\Pizza\Ideal\AdobeStock_57849082.jpeg: 448x640 1 pizza, 3623.0ms
Speed: 5.1ms preprocess, 3623.0ms inference, 7.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 C:\Users\aliat\Downloads\Aithon\Ads_problem4\Pizza\Other\AdobeStock_286178925.jpeg: 448x640 3 bowls, 1 sandwich, 1 hot dog, 1 pizza, 6 donuts, 3510.2ms
Speed: 3.0ms preprocess, 3510.2ms inference, 7.5ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 C:\Users\aliat\Downloads\Aithon\Ads_problem4\Pizza\Other\AdobeStock_315333860.jpeg: 448x640 4 persons, 1 wine glass, 1 cup, 1 fork, 1 bowl, 5 pizzas, 1 dining table, 3710.3ms
Speed: 6.0ms preprocess, 3710.3ms inference, 4.2ms postprocess per image at 

{'Ads_problem4\\Pizza\\Ideal\\AdobeStock_323628715.jpeg': {'pizza': ['4', [153.98963928222656, 291.21673583984375, 226.3890838623047, 233.55429077148438], [720.8762817382812, 239.23458862304688, 237.6578369140625, 234.1356201171875], [420.0894775390625, 421.43365478515625, 226.013427734375, 223.06881713867188], [453.09600830078125, 159.75413513183594, 249.83035278320312, 261.5409851074219]], 'dining table': ['1', [441.50677490234375, 292.5229187011719, 872.79833984375, 583.9114379882812]], 'person': ['3', [245.96791076660156, 501.9630126953125, 274.6285095214844, 173.05477905273438], [510.864990234375, 99.07861328125, 362.390625, 195.84207153320312], [84.78338623046875, 113.57682800292969, 108.3873291015625, 226.08212280273438]], 'cup': ['3', [634.4913330078125, 131.7866973876953, 82.74188232421875, 83.9341812133789], [272.84417724609375, 172.73043823242188, 87.26675415039062, 81.94517517089844], [85.76455688476562, 477.207275390625, 91.4722671508789, 84.1722412109375]]}, 'Ads_problem4

In [4]:
rev_name = {}
for i,j in names.items():
    rev_name[j] = i


In [128]:
ideal_dict = {}
for main,sub in data.items():
    ideal_ent = []
    key = list(sub.keys())[0]
    for img in sub[key]:
        ideal_ent.extend(list(entity[img].items()))
    ideal_dict[key] = dict(ideal_ent)
    

In [129]:
other_dict =  {}
for main,sub in data.items():
    other_ent = []
    key = list(sub.keys())[1]
    for img in sub[key]:
        other_ent.extend(list(entity[img].items()))
    other_dict[key] = dict(other_ent)
    

In [130]:
count_dict = {
    "Ideal":ideal_dict,
    "Other":other_dict
}


In [131]:
import csv


for pth,cnt in count_dict["Ideal"].items():
    key = (pth[:-6])
    for k in data[key]:
        
        if (k.endswith("Other\\")):
            img_other= (data[key][k])
            for img in img_other:
                
                other_ent = (dict(entity[img].items()))
                file = (img.split(".")[0])
                print("-"*50)
                print(img)
                missing,extra,extra_ent,missing_ent = [],[],[],[]
                for ent1,cnt1 in cnt.items():
                    if ent1 not in list(other_ent.keys()):
                        print(f"Missing {ent1}(class-{rev_name[ent1]})")
                        missing.append(f"Missing {ent1}(class-{rev_name[ent1]})")
                        missing_ent.append(ent1)
                for ent2,cnt2 in other_ent.items():
                    if ent2 not in list(cnt.keys()):
                        print(f"Extra {ent2}(class-{rev_name[ent2]})")
                        extra.append(f"Extra {ent2}(class-{rev_name[ent2]})")
                        extra_ent.append(ent2)
                ents = missing+extra
                coords = []
                mc,ec = [],[]
                
                for i in missing_ent:
                    mc.append(ideal_dict[pth][i][1:])
                        
                
                for i in extra_ent:
                    ec.append(entity[img][i][1:])
                
                coord = mc + ec
                
                values = []
                for i in range(len(ents)):
                    
                    values.append(["Entity",ents[i],coord[i]])
                f = open(f"{file}.csv","w+")
                cw = csv.writer(f)
                header = ["Entity","Difference","Meta"]
                cw.writerow(header)
                cw.writerows(values)
                f.close()


--------------------------------------------------
Ads_problem4\Pizza\Other\AdobeStock_286178925.jpeg
Missing dining table(class-60)
Missing person(class-0)
Missing cup(class-41)
Extra donut(class-54)
Extra sandwich(class-48)
Extra bowl(class-45)
Extra hot dog(class-52)
--------------------------------------------------
Ads_problem4\Pizza\Other\AdobeStock_315333860.jpeg
Extra fork(class-42)
Extra bowl(class-45)
Extra wine glass(class-40)
--------------------------------------------------
Ads_problem4\Pizza\Other\AdobeStock_602492718.jpeg
Missing dining table(class-60)
Missing person(class-0)
Missing cup(class-41)
--------------------------------------------------
Ads_problem4\Ramen\Other\AdobeStock_175017150.jpeg
Missing bowl(class-45)
Missing spoon(class-44)
Missing broccoli(class-50)
Extra bench(class-13)
--------------------------------------------------
Ads_problem4\Ramen\Other\AdobeStock_288691583.jpeg
Missing spoon(class-44)
Missing broccoli(class-50)
----------------------------